<a href="https://colab.research.google.com/github/wanghsinche/name-pls.js/blob/notebook/notebook/sklearn2onnx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle
f = open('/content/drive/MyDrive/Colab Notebooks/R100_Glint360K_XSList.pkl','rb')
represent = pickle.load(f)
f.close()

In [2]:
import re

Y = []
X = []
threshold = 3000
idx = 0
for idt in represent:
    if idx > threshold:
        break
    idx += 1
    Y.append(re.sub(r'_\d+.jpg','',idt))
    X.append(represent[idt])
print(len(Y))
print(len(X))

2909
2909


In [3]:
f = open('/content/drive/MyDrive/Colab Notebooks/insightface/github-represent.pkl', 'rb')
githubRep = pickle.load(f)
f.close()
import numpy as np
X = np.concatenate((X,githubRep['feat']), axis=0)
Y = np.concatenate((Y,githubRep['target']), axis=0)

print(X.shape)
print(Y.shape)
print(githubRep['feat'].shape)

(2937, 512)
(2937,)
(28, 512)


In [4]:
from random import randint
z = dict()


X_train = []
X_test = []
y_train = []
y_test = []

id = 0

for x, y in zip(X, Y):
    
    count = z.get(y, 0)

    if count <6:
        X_train.append(x)
        y_train.append(y)
    elif randint(1,10)>5:
        X_train.append(x)
        y_train.append(y)
    else:
        X_test.append(x)
        y_test.append(y)
    
    z.update({y:count+1})

print(len(X_train))


2548


In [15]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

clf = BernoulliNB()


y_pred = clf.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d"% (len(X_test), (y_test != y_pred).sum()))


Number of mislabeled points out of a total 389 points : 97


In [6]:
!pip install skl2onnx 

     |████████████████████████████████| 271 kB 5.4 MB/s 
     |████████████████████████████████| 12.7 MB 27.6 MB/s 
     |████████████████████████████████| 78 kB 6.9 MB/s 


In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType, Int32TensorType, StringTensorType

initial_type = [('float_input', FloatTensorType([None, 512]))]
final_types = [('label', StringTensorType([None, 1])), ('probabilities', FloatTensorType([None, 384]))]

# label', 'probabilities
# final_types = [('int_output', Int32TensorType([None, 1]))]


options = {BernoulliNB: {'zipmap': False}} # enables getting probabilities in Node.

onx = convert_sklearn(clf, initial_types=initial_type, options=options)
with open("bernoullinb_R100_Glint360K_XSList.onnx", "wb") as f:
    f.write(onx.SerializeToString())


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `coef_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `intercept_` was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:103: FutureWarning: Attribute `n_features_` was deprecated in version 1.0 and will be removed in 1.2. Use `n_features_in_` instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
type(clf)

sklearn.naive_bayes.BernoulliNB

In [9]:
!pip install onnxruntime

     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [10]:
import onnxruntime as rt
import numpy

ii = [X_test[0]]

sess = rt.InferenceSession("bernoullinb_R100_Glint360K_XSList.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

sess.run([label_name], {input_name: ii})[0]


array(['129251/Kokona_Asakura'], dtype=object)

In [11]:
clf.predict(ii)

array(['129251/Kokona_Asakura'], dtype='<U24')

In [12]:
namedict = dict()
idx = 0
for i in Y:
  if i not in namedict:
    namedict.update({i:idx})
    idx += 1

In [13]:
import json 
f = open('label-id.json','w')
json.dump(namedict, f)
f.close()

In [14]:
import json 
f = open('classes.json','w')
json.dump(list(clf.classes_), f)
f.close()